In [39]:
# Importing necessary packages
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Loading dataset into a DataFrame
col_names = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "native_country", "income"]

# Reading CSV, handling missing value token " ?", and trimming spaces after delimiters
df = pd.read_csv("/Users/Biadela/Documents/Data Analysis:Science/Codveda Internship/codveda-data-science/adult.csv", header = None, names = col_names, na_values = "?", skipinitialspace = True)

print("The dataset is of the shape " + str(df.shape))
df.head()

The dataset is of the shape (32561, 15)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [40]:
# Inspecting data types and missing values
print("The dataset consists of:")
df.info()

print("\nThe number of missing values per column are:")
print(df.isnull().sum().sort_values(ascending = False))

print("\nThe number of unique elements per object column is:")
print(df.select_dtypes(include = "object").nunique().sort_values(ascending = False))

The dataset consists of:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  31978 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB

The number of missing values per column are:
occupation        1843

In [41]:
# Removing rows with missing values
df = df.dropna().reset_index(drop = True)

print(f"\nRows remaining after dropping missing values: {df.shape[0]}")

print(df.isnull().sum())


Rows remaining after dropping missing values: 30162
age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income            0
dtype: int64


In [42]:
# Checking for and dealing with duplicates
dups = df.duplicated().sum()
print("Duplicate rows: ", dups)

if dups > 0:
    df = df.drop_duplicates().reset_index(drop = True)
    print("Duplicates dropped, new shape is ", df.shape)
else:
    print("No duplicates to drop")

Duplicate rows:  23
Duplicates dropped, new shape is  (30139, 15)


In [43]:
# Detecting and clipping outliers using IQR
cols_for_outliers = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]

outlier_summary = {}
for col in cols_for_outliers:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - (1.5 * IQR)
    upper = Q3 + (1.5 * IQR)
    num_outliers = ((df[col] < lower) | (df[col] > upper)).sum()
    outlier_summary[col] = {"Q1": int(Q1), "Q3": int(Q3), "IQR": int(IQR), "lower": int(lower), "upper": int(upper), "num_outliers": int(num_outliers)}
    df[col] = df[col].clip(lower = lower, upper = upper)

import pprint
pprint.pprint(outlier_summary)
print("\nAfter clipping, the ranges are: ")
print(df[cols_for_outliers].agg(["min", "max", "median"]))

{'age': {'IQR': 19,
         'Q1': 28,
         'Q3': 47,
         'lower': 0,
         'num_outliers': 168,
         'upper': 75},
 'capital_gain': {'IQR': 0,
                  'Q1': 0,
                  'Q3': 0,
                  'lower': 0,
                  'num_outliers': 2538,
                  'upper': 0},
 'capital_loss': {'IQR': 0,
                  'Q1': 0,
                  'Q3': 0,
                  'lower': 0,
                  'num_outliers': 1427,
                  'upper': 0},
 'education_num': {'IQR': 4,
                   'Q1': 9,
                   'Q3': 13,
                   'lower': 3,
                   'num_outliers': 193,
                   'upper': 19},
 'fnlwgt': {'IQR': 119977,
            'Q1': 117627,
            'Q3': 237604,
            'lower': -62338,
            'num_outliers': 904,
            'upper': 417570},
 'hours_per_week': {'IQR': 5,
                    'Q1': 40,
                    'Q3': 45,
                    'lower': 32,
                  

In [44]:
# Engineering new features
df["capital_net"] = df["capital_gain"] - df["capital_loss"]

age_bins = [0, 25, 35, 45, 55, 65, 125]
age_labels = ["<25", "25-34", "35-44", "45-54", "55-64", "65+"]
df["age_group"] = pd.cut(df["age"], bins = age_bins, labels = age_labels, right = False)

print(df[["age", "age_group", "capital_gain", "capital_loss", "capital_net"]].head())

    age age_group  capital_gain  capital_loss  capital_net
0  39.0     35-44             0             0            0
1  50.0     45-54             0             0            0
2  38.0     35-44             0             0            0
3  53.0     45-54             0             0            0
4  28.0     25-34             0             0            0


In [45]:
# Preparing features (X) and targets (y)
y = df["income"].apply(lambda s: 1 if ">" in s else 0).astype(int)
X = df.drop(columns = ["income"])

print("X shape: ", X.shape, "y shape: ", y.shape)
print("Target distribution:\n", y.value_counts(normalize = True))

X shape:  (30139, 16) y shape:  (30139,)
Target distribution:
 income
0    0.750954
1    0.249046
Name: proportion, dtype: float64


In [49]:
# Column lists for the transformer
numeric_cols = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week", "capital_net"]
categorical_cols = [c for c in X.select_dtypes(include = "object").columns.tolist() if c not in ["income"]]
categorical_cols = [c for c in categorical_cols if c != "age_group"] + ["age_group"]

print("Numeric columns: ", numeric_cols)
print("Categorical columns (sample): ", categorical_cols[:10])

# Building ColumnTransformer
preprocessor = ColumnTransformer(transformers = [("num", StandardScaler(), numeric_cols), ("cat", OneHotEncoder(handle_unknown = "ignore", sparse_output = False), categorical_cols)], remainder = "drop")

X_prepared = preprocessor.fit_transform(X)

num_feature_names = numeric_cols
cat_feature_names = preprocessor.named_transformers_["cat"].get_feature_names_out(categorical_cols)
feature_names = list(num_feature_names) + list(cat_feature_names)

print("Prepared shape: ", X_prepared.shape)
print("Number of features after one-hot encoding: ", len(feature_names))

Numeric columns:  ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week', 'capital_net']
Categorical columns (sample):  ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'age_group']
Prepared shape:  (30139, 111)
Number of features after one-hot encoding:  111


In [52]:
# Putting transformed data back into a DataFrame and saving
X_processed = pd.DataFrame(X_prepared, columns = feature_names)
X_processed["income"] = y.values

print("Processed dataset shape: ", X_processed.shape)
display(X_processed.head())

X_processed.to_csv("adult_preprocessed.csv", index = False)

Processed dataset shape:  (30139, 112)


,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,capital_net,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia,age_group_25-34,age_group_35-44,age_group_45-54,age_group_55-64,age_group_65+,age_group_<25,income
0,0.045401,-1.147731,1.136712,0.0,0.0,-0.231097,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
1,0.889957,-1.086920,1.136712,0.0,0.0,-1.451391,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,-0.031377,0.301753,-0.447774,0.0,0.0,-0.231097,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
3,1.120291,0.501919,-1.240017,0.0,0.0,-0.231097,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
4,-0.799155,1.589982,1.136712,0.0,0.0,-0.231097,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
